# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [30]:
import pandas as pd
import numpy as np
from scipy import stats

In [31]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [32]:
# how many values? 
len(df)

8188

In [33]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [34]:
# how many people were tested in each group?
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [35]:
# what are the choices a user can make?
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [36]:
# group by control/experiment group and sum actions per group
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [37]:
#Your code here
# Comparing the # of clicks for the experimental home page to the control 
# Will use Welch's t-test to compare amount of clicks between two pages
# Null: There is no difference in the amount of clicks for the homepage and experimental page
# Alt: People who saw the alternative page will click more than the control page. 

In [38]:
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [39]:
# create a freq_count column that sums up views and clicks
df.groupby('group')['action'].count()

group
control       4264
experiment    3924
Name: action, dtype: int64

In [40]:
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [41]:
# create a new column 'control_click', where 1 assignment if click, 0 for view
df.loc[(df['group'] == 'control') & (df['action'] == 'click'),'control_click'] = 1
df.loc[(df['group'] == 'control') & (df['action'] == 'view'),'control_click'] = 0

In [42]:
# double check control_click totals
df['control_click'].value_counts()

0.0    3332
1.0     932
Name: control_click, dtype: int64

In [43]:
# create a new column 'exp_click', where 1 assignment if click, 0 for view
df.loc[(df['group'] == 'experiment') & (df['action'] == 'click'),'exp_click'] = 1
df.loc[(df['group'] == 'experiment') & (df['action'] == 'view'),'exp_click'] = 0

In [44]:
# fill NaNs with 0
df['control_click'].fillna(0, inplace = True)
df['exp_click'].fillna(0, inplace = True)

In [45]:
# double check experiment counts
df['exp_click'].value_counts()

0.0    7260
1.0     928
Name: exp_click, dtype: int64

In [46]:
# total amount of people who viewed in control
total_control_sample =  3332
print('Total Control Views:', total_control_sample)

# total clicks on control group
total_control_clicks = df['control_click'].sum()
print('Total Control Clicks:', total_control_clicks)

print('')

# total amount of people who viewed in exp
total_exp_sample = 2996
print('Total Experiment Views:', total_exp_sample)

# total clicks on experiment group
total_exp_clicks = df['exp_click'].sum()
print('Total Experiment Clicks:',total_exp_clicks)

print('')

# Calculate Click Rates
control_click_rate = round(total_control_clicks / total_control_sample, 5)
exp_click_rate = round(total_exp_clicks / total_exp_sample, 5)

print('Control Click Rate:', control_click_rate)
print('Experiment Click Rate:', exp_click_rate)

# Difference in click rates
print('Difference in Click Rate:', round(exp_click_rate - control_click_rate, 5),"%")

Total Control Views: 3332
Total Control Clicks: 932.0

Total Experiment Views: 2996
Total Experiment Clicks: 928.0

Control Click Rate: 0.27971
Experiment Click Rate: 0.30975
Difference in Click Rate: 0.03004 %


In [47]:
df

,timestamp,id,group,action,control_click,exp_click
0,2016-09-24 17:42:27.839496,804196,experiment,view,0.0,0.0
1,2016-09-24 19:19:03.542569,434745,experiment,view,0.0,0.0
2,2016-09-24 19:36:00.944135,507599,experiment,view,0.0,0.0
3,2016-09-24 19:59:02.646620,671993,control,view,0.0,0.0
4,2016-09-24 20:26:14.466886,536734,experiment,view,0.0,0.0
...,...,...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view,0.0,0.0
8184,2017-01-18 09:42:12.844575,755912,experiment,view,0.0,0.0
8185,2017-01-18 10:01:09.026482,458115,experiment,view,0.0,0.0
8186,2017-01-18 10:08:51.588469,505451,control,view,0.0,0.0


In [48]:
# determine if difference in click rate is significant or not
# use welch t test , ie. set equal variance to False

stats.ttest_ind(df['control_click'], df['exp_click'], equal_var = False)

Ttest_indResult(statistic=0.098504838012413, pvalue=0.9215326362459715)

In [49]:
# since p value is above the alpha = 0.05, we fail to reject the null hypothesis. 

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [50]:
#Your code here
expected_exp_clicks = control_click_rate * total_exp_sample
expected_exp_clicks

838.01116

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [51]:
#Your code here
n = total_exp_sample
p = control_click_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.56849727672411


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [52]:
# find z score
z_score = (total_exp_clicks - expected_exp_clicks) / std
z_score

3.662773469065782

In [55]:
#Your code here
stats.norm.sf(abs(z_score))

0.0001247495831401237

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.